Redis DB:

  1)Use the same SQL data as in MongoDB.
  
  2)Create Redis DB, with the data from SQL:
    Set a key with a specific value in Redis.
    Retrieve the value of a key from Redis.
    Delete a key-value pair from Redis.

Getting and formatting data from SQL

In [1]:
import mysql.connector

try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='1234',
        database='advanced_sql')

    if connection.is_connected():
        print('Connected to MySQL database!')
    else:
        print('Failed to connect to the database.')
except mysql.connector.Error as e:
    print(f'Error connecting to MySQL: {e}')

Connected to MySQL database!


In [2]:
def execute_query(query):
    cursor = connection.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows

In [3]:
from decimal import Decimal
from datetime import date

def convert_value(value):
    if isinstance(value, Decimal):
        return float(value)
    elif isinstance(value, date):
        return value.strftime('%Y-%m-%d')
    return value

In [4]:
query = """SELECT * FROM employees"""
result = execute_query(query)
for row in result:
    formatted_row = [convert_value(value) for value in row]
    print(formatted_row)

[1, 'John Doe', 35, 47655.0, 'Sales']
[2, 'Jane Smith', 28, 37506.0, 'HR']
[3, 'Mike Johnson', 42, 34947.0, 'Finance']
[4, 'Emily Brown', 29, 68342.0, 'IT']
[5, 'David Lee', 38, 48759.0, 'Sales']


Creating Redis DB

In [9]:
import redis

redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

In [10]:
def save_data_to_redis(data):
    for row in data:
        user_id = int(row[0])
        key = f'user:{user_id}' # Set a key with a user_id value in Redis

        formatted_row = [convert_value(value) for value in row[1:]]

        # dictionary with field names as keys and formatted values
        value = {field: formatted_row[i] for i, field in enumerate(['name', 'age', 'salary', 'department'])}

        redis_client.hmset(key, value)

save_data_to_redis(result)

C:\Users\dovil\AppData\Local\Temp\ipykernel_14596\1847007115.py:11: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  redis_client.hmset(key, value)


Setting a key (user_id) in Redis and retrieving data from Redis

In [10]:
def get_user_info(user_id):
    key = f'user:{user_id}'
    user_info = redis_client.hgetall(key)
    return user_info

In [11]:
user_id = 2
user_info = get_user_info(user_id)
print(f'User with ID {user_id}: {user_info}')

User with ID 2 : {'name': 'Jane Smith', 'age': '28', 'salary': '37506.0', 'department': 'HR'}


Deleting data from Redis

In [12]:
def delete_user(user_id):
    key = f'user:{user_id}'
    redis_client.delete(key)

In [13]:
user_id = 2
delete_user(user_id)
print(f'User with ID {user_id} has been deleted')

User with ID 2 has been deleted
